In [1]:
from gensim.models import Word2Vec

In [2]:
import numpy as np
import _pickle as pickle

In [3]:
code_seq_2013 = pickle.load(open("../../DATA/code_seq_2013","rb"))

proc2cat = pickle.load(open("../../DATA/CCS/proc2cat","rb"))
diag2cat = pickle.load(open("../../DATA/CCS/diag2cat","rb"))
drug2cat = pickle.load(open("../../DATA/CCS/drug2cat","rb"))

# patient-level sentences

In [27]:
def get_ccs(seq, diag2cat, proc2cat, drug2cat):
    new_seq = []
    for p in seq:
        new_p = []
        for v in p:
            new_v = []
            for c in v:
                if c in diag2cat :
                    new_c = diag2cat[c]
                elif c in proc2cat:
                    new_c = proc2cat[c]
                elif c in drug2cat:
                    new_c = drug2cat[c]
                else:
                    new_c = c[:5]
                new_v.append(new_c)
            new_p.append(new_v)
        new_seq.append(new_p)
    return new_seq

def process_code(seq, PAD=True):
    new_seq = []
    if PAD: vocab2int = {"PAD":0}
    else: vocab2int = {}
    for p in seq:
        new_p = []
        for v in p:
            new_v = []
            for c in v:
                if c not in vocab2int: vocab2int[c] = len(vocab2int)
                new_v.append(vocab2int[c])
            new_p.append(new_v)
        new_seq.append(new_p)
    return new_seq, vocab2int

In [28]:
ccs_seq = get_ccs(code_seq_2013, diag2cat, proc2cat, drug2cat)
ccs_feature, ccs2int = process_code(ccs_seq)  
code_feature, code2int = process_code(code_seq_2013)  

In [29]:
ccs_sentence = [[c for v in p for c in v] for p in ccs_seq]

embed_size = 100
model_ccs = Word2Vec(ccs_sentence, size=embed_size, window=10, min_count=0, workers=4)

In [30]:
code_sentence = [[c for v in p for c in v] for p in code_seq_2013]

embed_size = 100
model_code = Word2Vec(code_sentence, size=embed_size, window=10, min_count=0, workers=4)

In [31]:
# code_embedding = np.zeros((len(code2int), embed_size))
# for code in code2int:
#     if code=="PAD": continue
#     code_embedding[code2int[code]] = model_code.wv[code]
# code_embedding = np.array(code_embedding)
# code_embedding.shape

In [32]:
# ccs_embedding = np.zeros((len(ccs2int), embed_size))
# for ccs in ccs2int:
#     if ccs=="PAD": continue
#     ccs_embedding[ccs2int[ccs]] = model_ccs.wv[ccs]
# ccs_embedding = np.array(ccs_embedding)
# ccs_embedding.shape

In [33]:
model_ccs.save("model_ccs")

In [34]:
model_code.save("model_code")